# Semana 2 - Desafio de Qualidade

Olá, me chamo Wesley e este é meu notebook que contém as resoluções para o desafio proposto na semana 2 da Aceleração PySpark na Capgemini.

O desafio consiste em garantir a qualidade dos dados contidos em três datasets (Airports, Planes e Flights) que serão respectivamente resolvidos, juntos com os exercícios propostos para esta semana.

Por fim, para colocarmos as mãos na massa, é importante lembrar que neste notebook será majoritariamente feito uso do PySpark, logo não serão inclusas as bibliotecas de data science do python (pandas, numpy, etc) que constam como padrão na [documentação oficial] do PySpark, pois o intuito deste notebook é aprender, e reforçar o que foi aprendido até o momento, da programação com o PySpark. 

[documentação oficial]: https://spark.apache.org/docs/latest/api/python/getting_started/install.html

## Vamos lá!

começaremos com o setup

In [1]:
#installing the required packages
!pip install pyspark
!pip install findspark
!pip install py4j

You should consider upgrading via the '/opt/python/envs/default/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/python/envs/default/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/python/envs/default/bin/python -m pip install --upgrade pip' command.


In [43]:
import findspark
findspark.init()
# PySpark is the Spark API for Python. In this lab, we use PySpark to initialize the spark context. 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [44]:
import re
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

In [47]:
# Expressoes regulares comuns
REGEX_ALPHA    = r'[a-zA-Z]+'
REGEX_INTEGER  = r'[0-9]+'
REGEX_FLOAT    = r'[0-9]+\.[0-9]+'
REGEX_ALPHANUM = r'[0-9a-zA-Z]+'
REGEX_EMPTY_STR= r'[\t ]+$'
REGEX_SPECIAL  = r'[!@#$%&*\(\)_]+'
REGEX_NNUMBER  = r'^N[1-9][0-9]{2,3}([ABCDEFGHJKLMNPRSTUVXWYZ]{1,2})'
REGEX_NNUMBER_INVALID = r'(N0.*$)|(.*[IO].*)'
REGEX_TIME_FMT = r'^(([0-1]?[0-9])|(2[0-3]))([0-5][0-9])$'

In [48]:
import re
#helper functions
def check_empty_column(col):
    return (F.col(col).isNull() | (F.col(col) == '') | F.col(col).rlike(REGEX_EMPTY_STR))

def check_column_range(col, from_value, to_value, leftInclusive = False, rightIncluse = False):
    check_range_expression = "(" + str(F.col(col)) + (" >= " if leftInclusive else " > ") + str(from_value) + " and " \
                                 + str(F.col(col)) + (" <= " if rightIncluse  else " < ") +")" 
    return eval(check_range_expression)
    
def create_regex_from_list(_list):
    return r'|'.join(map(lambda x : f".*({x}).*", _list))

tailnum_chars = F.udf(lambda value: ''.join([c for c in value if not c.isdigit()])[1:])

@F.udf
def get_byregex_and_extract(r_pattern, col_to_search):
    return (getattr(re.search(r_pattern, F.col(col_to_search), re.IGNORECASE), 'groups', lambda:[""])()[0].upper())

getbyreg = F.udf(lambda r_pattern,col_to_search: getattr(re.search(r_pattern, col_to_search, re.IGNORECASE), 'groups', lambda:[u""])()[0].upper())

In [3]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
        .builder \
        .appName("Quality Assurance Challenge") \
        .getOrCreate()

Essa será a configuração inicial para que possamos trabalhar os datasets nos exercícios a seguir. 

Os datasets Airports, Planes e Flights serão apresentados nesta ordem junto com seus respectivos exercícios. Caso haja dúvidas, não deixe de olhar os glossários que acompanham cada dataset para auxiliá-lo.

# Airports Dataset

![Airport](attachment:./images/airplanes_snapshot.png "a Crowded airport")
<p style="text-align: center;"> <a href="./glossary/airport_glossary.txt"> Airport Glossary </a> </p>


Este dataset contém 7 tarefas que serão trabalhadas a seguir, mas primeiro, iremos importar nosso dataset abaixo:

In [4]:
#lets describe our file
file = "./datasets/airports.csv"

#import the classes to make the schema
from pyspark.sql.types import StructType, IntegerType, FloatType, StringType, StructField

#set our schema (that you can see on glossary)
schema = StructType([
    StructField("faa",  StringType()),
    StructField("name", StringType()),
    StructField("lat",  FloatType()),
    StructField("lon",  FloatType()),
    StructField("alt",  IntegerType()),
    StructField("tz",   FloatType()),
    StructField("dst",  StringType())
])

#and create a new dataframe 
airport_df = spark \
            .read.format("csv") \
            .option("inferSchema", "false") \
            .option("header", "True")\
            .schema(schema)\
            .csv(file)

#lets take the rdd to work with too
airport_rdd = airport_df.rdd

#don't forget to create a view (important to work with spark sql)
airport_df.createOrReplaceTempView('airports_view')

#see how great is this
airport_df.printSchema()
display(airport_rdd.collect())

root
 |-- faa: string (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: float (nullable = true)
 |-- lon: float (nullable = true)
 |-- alt: integer (nullable = true)
 |-- tz: float (nullable = true)
 |-- dst: string (nullable = true)



[Row(faa='04G', name='Lansdowne Airport', lat=41.13047409057617, lon=-80.61958312988281, alt=1044, tz=-5.0, dst='A'),
 Row(faa='06A', name='Moton Field Municipal Airport', lat=32.4605712890625, lon=-85.6800308227539, alt=264, tz=-5.0, dst='A'),
 Row(faa='06C', name='Schaumburg Regional', lat=41.989341735839844, lon=-88.10124206542969, alt=801, tz=-6.0, dst='A'),
 Row(faa='06N', name='Randall Airport', lat=41.43191146850586, lon=-74.39156341552734, alt=523, tz=-5.0, dst='A'),
 Row(faa='09J', name='Jekyll Island Airport', lat=31.074472427368164, lon=-81.42778015136719, alt=11, tz=-4.0, dst='A'),
 Row(faa='0A9', name='Elizabethton Municipal Airport', lat=36.37122344970703, lon=-82.17341613769531, alt=1593, tz=-4.0, dst='A'),
 Row(faa='0G6', name='Williams County Airport', lat=41.46730422973633, lon=-84.50677490234375, alt=730, tz=-5.0, dst='A'),
 Row(faa='0G7', name='Finger Lakes Regional Airport', lat=42.88356399536133, lon=-76.78123474121094, alt=492, tz=-5.0, dst='A'),
 Row(faa='0P2', 

e nosso dataframe já está pronto para trabalharmos com ele! 
Vamos seguir com as tarefas: 

In [5]:
from pyspark.sql.functions import col, when, length
import re
#with sql
qa_faa_df = spark.sql("""
    select
        *,
        case
            when 
               faa is null or faa like '' then 'M'
            when 
               faa rlike '^[a-zA-Z ]*$' 
               or faa rlike '^([^0-9]*)$'
               and length(faa) between 3 and 5 then 'F'
            else
               faa
        end as qa_faa
    from airports_view
    """)

qa_faa_df.show()

#and with the rdd
faa_airport_rdd = airport_df.select('faa').rdd.map(lambda el: "M" if el[0] == None or el[0] == "" else ("F" if ((len(el[0]) < 3 and len(el[0]) > 5) or not re.match('^[^0-9a-zA-Z]+$',el[0])) else el[0] ))
faa_airport_rdd.collect()

airport_df = airport_df.withColumn("qa_faa", 
                    when((col("faa") == "") |
                    (col("faa").isNull())   |
                    (col('faa').rlike('\t') |
                    (col('faa').rlike(' +')))     , "M")\
                    .when(
                        (length("faa") < 3) & 
                        (length("faa") > 5) &
                        (col('faa').rlike('^[a-zA-Z ]*$')) | (col('faa').rlike('^([^0-9]*)$')), "F"
                    )\
                    .otherwise(col('faa'))
                )
airport_df.select('qa_faa').show()

+---+--------------------+---------+-----------+----+----+---+------+
|faa|                name|      lat|        lon| alt|  tz|dst|qa_faa|
+---+--------------------+---------+-----------+----+----+---+------+
|04G|   Lansdowne Airport|41.130474|  -80.61958|1044|-5.0|  A|   04G|
|06A|Moton Field Munic...| 32.46057|  -85.68003| 264|-5.0|  A|   06A|
|06C| Schaumburg Regional| 41.98934|  -88.10124| 801|-6.0|  A|   06C|
|06N|     Randall Airport| 41.43191|  -74.39156| 523|-5.0|  A|   06N|
|09J|Jekyll Island Air...|31.074472|  -81.42778|  11|-4.0|  A|   09J|
|0A9|Elizabethton Muni...|36.371223| -82.173416|1593|-4.0|  A|   0A9|
|0G6|Williams County A...|41.467304| -84.506775| 730|-5.0|  A|   0G6|
|0G7|Finger Lakes Regi...|42.883564| -76.781235| 492|-5.0|  A|   0G7|
|0P2|Shoestring Aviati...|39.794823| -76.647194|1000|-5.0|  U|   0P2|
|0S9|Jefferson County ...| 48.05381|-122.810646| 108|-8.0|  A|   0S9|
|0W3|Harford County Ai...|39.566837|   -76.2024| 409|-5.0|  A|   0W3|
|10C|  Galt Field Ai

oi

In [6]:
#with sql
qa_name_df = spark.sql("""
    select
        *,
        case
            when 
               name is null or faa like '' then 'M'
            else
               name
        end as qa_name
    from airports_view
    """)
qa_name_df.show()

#and with rdd
def qa_name_mapping(el):
    if not el or el == "":
        return "M"
    else:
        return el
result = airport_rdd.map(lambda el: qa_name_mapping(el['name'])).collect()
print(result)

airport_df = airport_df.withColumn("qa_name", 
                    when((col("name") == "")     |
                        (col("name").isNull())  |
                        (col('faa').rlike('\t') |
                        (col('faa').rlike(' +'))), "M")\
                    .otherwise(col('name'))
                )
airport_df.select('qa_name').show()

+---+--------------------+---------+-----------+----+----+---+--------------------+
|faa|                name|      lat|        lon| alt|  tz|dst|             qa_name|
+---+--------------------+---------+-----------+----+----+---+--------------------+
|04G|   Lansdowne Airport|41.130474|  -80.61958|1044|-5.0|  A|   Lansdowne Airport|
|06A|Moton Field Munic...| 32.46057|  -85.68003| 264|-5.0|  A|Moton Field Munic...|
|06C| Schaumburg Regional| 41.98934|  -88.10124| 801|-6.0|  A| Schaumburg Regional|
|06N|     Randall Airport| 41.43191|  -74.39156| 523|-5.0|  A|     Randall Airport|
|09J|Jekyll Island Air...|31.074472|  -81.42778|  11|-4.0|  A|Jekyll Island Air...|
|0A9|Elizabethton Muni...|36.371223| -82.173416|1593|-4.0|  A|Elizabethton Muni...|
|0G6|Williams County A...|41.467304| -84.506775| 730|-5.0|  A|Williams County A...|
|0G7|Finger Lakes Regi...|42.883564| -76.781235| 492|-5.0|  A|Finger Lakes Regi...|
|0P2|Shoestring Aviati...|39.794823| -76.647194|1000|-5.0|  U|Shoestring Avi

Curiosidade

In [7]:
#with sql
qa_lat_df = spark.sql("""
    select
        *,
        case
            when 
               lat is null or faa like '' then 'M'
            when
               lat > 180.0 or lat < -180.0 then 'I'
            when
                lat rlike '/^\w*?[a-zA-Z]\w*$/' then 'A'
            else
               lat
        end as qa_lat
    from airports_view
    """)
qa_name_df.show()

#and with rdd
def qa_lat_mapping(el):
    if not el['lat'] or el['lat'] == "":
        return "M"
    elif el['lat'] > 180.0 and el['lat'] < -180.0:
        return 'I'
    elif not re.match('^([1-9]\d*(\.|\,)\d*|0?(\.|\,)\d*[1-9]\d*|[1-9]\d*)$',str(el['lat'])):
        return 'A'
    else:
        return el['lat']

result = airport_rdd.map(qa_lat_mapping).collect()
print(result)

airport_df = airport_df.withColumn("qa_lat", 
                    when((col("lat") == "")      |
                        (col("lat").isNull())   |
                        (col('lat').rlike('\t') |
                        (col('lat').rlike(' +'))), "M")\
                    .when(
                        (col("lat") > '180.0') & 
                        (col("lat") < '-180.0'), "I"
                    )\
                    .when((col('lat').rlike('[a-zA-Z ]')), "A")
                    .otherwise(col('lat'))
                )
airport_df.select('qa_lat').show()

+---+--------------------+---------+-----------+----+----+---+--------------------+
|faa|                name|      lat|        lon| alt|  tz|dst|             qa_name|
+---+--------------------+---------+-----------+----+----+---+--------------------+
|04G|   Lansdowne Airport|41.130474|  -80.61958|1044|-5.0|  A|   Lansdowne Airport|
|06A|Moton Field Munic...| 32.46057|  -85.68003| 264|-5.0|  A|Moton Field Munic...|
|06C| Schaumburg Regional| 41.98934|  -88.10124| 801|-6.0|  A| Schaumburg Regional|
|06N|     Randall Airport| 41.43191|  -74.39156| 523|-5.0|  A|     Randall Airport|
|09J|Jekyll Island Air...|31.074472|  -81.42778|  11|-4.0|  A|Jekyll Island Air...|
|0A9|Elizabethton Muni...|36.371223| -82.173416|1593|-4.0|  A|Elizabethton Muni...|
|0G6|Williams County A...|41.467304| -84.506775| 730|-5.0|  A|Williams County A...|
|0G7|Finger Lakes Regi...|42.883564| -76.781235| 492|-5.0|  A|Finger Lakes Regi...|
|0P2|Shoestring Aviati...|39.794823| -76.647194|1000|-5.0|  U|Shoestring Avi

em

In [8]:
#with sql
qa_lon_df = spark.sql("""
    select
        *,
        case
            when 
               lon is null or lon like '' then 'M'
            when
               lon > 180.0 or lon < -180.0 then 'I'
            when
                lon rlike '/^\w*?[a-zA-Z]\w*$/' then 'A'
            else
               lon
        end as qa_lon
    from airports_view
    """)
qa_lon_df.show()

airport_df = airport_df.withColumn("qa_lon", 
                    when((col("lon") == "")      |
                        (col("lon").isNull())   |
                        (col('lon').rlike('\t') |
                        (col('lon').rlike(' +'))), "M")\
                    .when(
                        (col("lon") > '180.0') & 
                        (col("lon") < '-180.0'), "I"
                    )\
                    .when((col('lon').rlike('[a-zA-Z ]')), "A")
                    .otherwise(col('lon'))
                )
airport_df.select('qa_lon').show()

+---+--------------------+---------+-----------+----+----+---+-----------+
|faa|                name|      lat|        lon| alt|  tz|dst|     qa_lon|
+---+--------------------+---------+-----------+----+----+---+-----------+
|04G|   Lansdowne Airport|41.130474|  -80.61958|1044|-5.0|  A|  -80.61958|
|06A|Moton Field Munic...| 32.46057|  -85.68003| 264|-5.0|  A|  -85.68003|
|06C| Schaumburg Regional| 41.98934|  -88.10124| 801|-6.0|  A|  -88.10124|
|06N|     Randall Airport| 41.43191|  -74.39156| 523|-5.0|  A|  -74.39156|
|09J|Jekyll Island Air...|31.074472|  -81.42778|  11|-4.0|  A|  -81.42778|
|0A9|Elizabethton Muni...|36.371223| -82.173416|1593|-4.0|  A| -82.173416|
|0G6|Williams County A...|41.467304| -84.506775| 730|-5.0|  A| -84.506775|
|0G7|Finger Lakes Regi...|42.883564| -76.781235| 492|-5.0|  A| -76.781235|
|0P2|Shoestring Aviati...|39.794823| -76.647194|1000|-5.0|  U| -76.647194|
|0S9|Jefferson County ...| 48.05381|-122.810646| 108|-8.0|  A|-122.810646|
|0W3|Harford County Ai...

pessoa

In [9]:
#with sql
qa_alt_df = spark.sql("""
    select
        *,
        case
            when 
               alt is null or alt like '' then 'M'
            when
               alt < 0 then 'I'
            when
               alt rlike '/^\w*?[a-zA-Z]\w*$/' then 'A'
            else
               alt
        end as qa_alt
    from airports_view
    """)
qa_alt_df.show()

airport_df = airport_df.withColumn("qa_alt", 
                     when((col("alt") == "")      |
                           (col("alt").isNull())   |
                           (col('alt').rlike('\t') |
                           (col('alt').rlike(' +'))), "M")\
                     .when((col("alt") < '0'), "I")\
                     .when( (col('alt').rlike('[a-zA-Z ]')), "A")
                     .otherwise(col('alt'))
               )
airport_df.show()

+---+--------------------+---------+-----------+----+----+---+------+
|faa|                name|      lat|        lon| alt|  tz|dst|qa_alt|
+---+--------------------+---------+-----------+----+----+---+------+
|04G|   Lansdowne Airport|41.130474|  -80.61958|1044|-5.0|  A|  1044|
|06A|Moton Field Munic...| 32.46057|  -85.68003| 264|-5.0|  A|   264|
|06C| Schaumburg Regional| 41.98934|  -88.10124| 801|-6.0|  A|   801|
|06N|     Randall Airport| 41.43191|  -74.39156| 523|-5.0|  A|   523|
|09J|Jekyll Island Air...|31.074472|  -81.42778|  11|-4.0|  A|    11|
|0A9|Elizabethton Muni...|36.371223| -82.173416|1593|-4.0|  A|  1593|
|0G6|Williams County A...|41.467304| -84.506775| 730|-5.0|  A|   730|
|0G7|Finger Lakes Regi...|42.883564| -76.781235| 492|-5.0|  A|   492|
|0P2|Shoestring Aviati...|39.794823| -76.647194|1000|-5.0|  U|  1000|
|0S9|Jefferson County ...| 48.05381|-122.810646| 108|-8.0|  A|   108|
|0W3|Harford County Ai...|39.566837|   -76.2024| 409|-5.0|  A|   409|
|10C|  Galt Field Ai

como

In [ ]:

airport_df.select('qa_dst').show()

In [10]:
#with sql
qa_tz_df = spark.sql("""
    select
        *,
        case
            when 
               tz is null or alt like '' then 'M'
            when
               tz <= -11 or tz >= 14 then 'I'
            when
               tz rlike '/^\w*?[a-zA-Z]\w*$/' then 'A'
            else
               tz
        end as qa_tz
    from airports_view
    """)
qa_tz_df.show()

airport_df =airport_df.withColumn("qa_tz", 
                  when((col("tz") == "")      |
                        (col("tz").isNull())   |
                        (col('tz').rlike('\t') |
                        (col('tz').rlike(' +')) ), "M")\
                  .when((col("tz") <= '-11.0') &
                        (col("tz") >= '14.0'), "I")\
                  .when( (col('tz').rlike('[a-zA-Z ]')), "A")
                  .otherwise(col('tz'))
            )
airport_df.select('qa_tz').show()

+---+--------------------+---------+-----------+----+----+---+-----+
|faa|                name|      lat|        lon| alt|  tz|dst|qa_tz|
+---+--------------------+---------+-----------+----+----+---+-----+
|04G|   Lansdowne Airport|41.130474|  -80.61958|1044|-5.0|  A| -5.0|
|06A|Moton Field Munic...| 32.46057|  -85.68003| 264|-5.0|  A| -5.0|
|06C| Schaumburg Regional| 41.98934|  -88.10124| 801|-6.0|  A| -6.0|
|06N|     Randall Airport| 41.43191|  -74.39156| 523|-5.0|  A| -5.0|
|09J|Jekyll Island Air...|31.074472|  -81.42778|  11|-4.0|  A| -4.0|
|0A9|Elizabethton Muni...|36.371223| -82.173416|1593|-4.0|  A| -4.0|
|0G6|Williams County A...|41.467304| -84.506775| 730|-5.0|  A| -5.0|
|0G7|Finger Lakes Regi...|42.883564| -76.781235| 492|-5.0|  A| -5.0|
|0P2|Shoestring Aviati...|39.794823| -76.647194|1000|-5.0|  U| -5.0|
|0S9|Jefferson County ...| 48.05381|-122.810646| 108|-8.0|  A| -8.0|
|0W3|Harford County Ai...|39.566837|   -76.2024| 409|-5.0|  A| -5.0|
|10C|  Galt Field Airport| 42.4028

está?

In [11]:
expected_categorys = ["E", "A", "S", "O", "Z", "N", "U"]

#with sql
qa_dst_df = spark.sql("""
    select
        *,
        case
            when 
                dst is null or dst like '' then 'M'
            when
                dst not in ('E', 'A', 'S', 'O', 'Z', 'N', 'U') then 'C'
            when
                dst rlike '^[0-9]*$' then 'N'
        end as qa_dst
    from airports_view
    """)

qa_dst_df.show()

airport_df =airport_df.withColumn("qa_dst", 
                when((col("dst") == "")      |
                    (col("dst").isNull())   |
                    (col('dst').rlike('\t') |
                    (col('dst').rlike(' +')) ), "M")\
                .when( (col('dst').rlike('^[0-9]*$') == True), "N")\
                .when((col("dst").isin(expected_categorys) == False) , "C")\
                .otherwise(col('dst'))
            )

+---+--------------------+---------+-----------+----+----+---+------+
|faa|                name|      lat|        lon| alt|  tz|dst|qa_dst|
+---+--------------------+---------+-----------+----+----+---+------+
|04G|   Lansdowne Airport|41.130474|  -80.61958|1044|-5.0|  A|  null|
|06A|Moton Field Munic...| 32.46057|  -85.68003| 264|-5.0|  A|  null|
|06C| Schaumburg Regional| 41.98934|  -88.10124| 801|-6.0|  A|  null|
|06N|     Randall Airport| 41.43191|  -74.39156| 523|-5.0|  A|  null|
|09J|Jekyll Island Air...|31.074472|  -81.42778|  11|-4.0|  A|  null|
|0A9|Elizabethton Muni...|36.371223| -82.173416|1593|-4.0|  A|  null|
|0G6|Williams County A...|41.467304| -84.506775| 730|-5.0|  A|  null|
|0G7|Finger Lakes Regi...|42.883564| -76.781235| 492|-5.0|  A|  null|
|0P2|Shoestring Aviati...|39.794823| -76.647194|1000|-5.0|  U|  null|
|0S9|Jefferson County ...| 48.05381|-122.810646| 108|-8.0|  A|  null|
|0W3|Harford County Ai...|39.566837|   -76.2024| 409|-5.0|  A|  null|
|10C|  Galt Field Ai

In [61]:
#Após feitas as transformações, vamos salvar nosso arquivo
airport_df.write.parquet("qa_outputs/airport_qa.parquet")
airport_df.show()

+---+--------------------+---------+-----------+----+----+---+------+--------------------+---------+-----------+------+-----+------+
|faa|                name|      lat|        lon| alt|  tz|dst|qa_faa|             qa_name|   qa_lat|     qa_lon|qa_alt|qa_tz|qa_dst|
+---+--------------------+---------+-----------+----+----+---+------+--------------------+---------+-----------+------+-----+------+
|04G|   Lansdowne Airport|41.130474|  -80.61958|1044|-5.0|  A|   04G|   Lansdowne Airport|41.130474|  -80.61958|  1044| -5.0|     A|
|06A|Moton Field Munic...| 32.46057|  -85.68003| 264|-5.0|  A|   06A|Moton Field Munic...| 32.46057|  -85.68003|   264| -5.0|     A|
|06C| Schaumburg Regional| 41.98934|  -88.10124| 801|-6.0|  A|   06C| Schaumburg Regional| 41.98934|  -88.10124|   801| -6.0|     A|
|06N|     Randall Airport| 41.43191|  -74.39156| 523|-5.0|  A|   06N|     Randall Airport| 41.43191|  -74.39156|   523| -5.0|     A|
|09J|Jekyll Island Air...|31.074472|  -81.42778|  11|-4.0|  A|   09J|

# Planes Dataset

![Airport](attachment:./images/planes_snapshot.png "an airport Hangar")
<p style="text-align: center;"> <a href="./glossary/planes_glossary.txt"> Planes Glossary </a> </p>

Precisamos configurar esse novo dataset pra trabalharmos:

In [13]:
#lets describe our new file
planes_file = "./datasets/planes.csv"

#set our schema (that you can see on glossary)
schema = StructType([
    StructField("tailnum",      StringType()),
    StructField("year",         IntegerType()),
    StructField("type",         StringType()),
    StructField("manufacturer", StringType()),
    StructField("model",   StringType()),
    StructField("engines", IntegerType()),
    StructField("seats",   IntegerType()),
    StructField("speed",   IntegerType()),
    StructField("engine",  StringType())
])

#and create a new dataframe 
planes_df = spark \
           .read.format("csv") \
           .option("inferSchema", "false") \
           .option("header", "True")\
           .schema(schema)\
           .csv(planes_file)

#lets take the rdd to work with too
planes_rdd = planes_df.rdd

#don't forget to create a view (important to work with spark sql)
planes_df.createOrReplaceTempView('planes_view')

Com nosso dataset configurado, estamos prontos para completar as tarefas a serem realizadas, que estão descritas a seguir :)

Você

In [14]:
#with sql
qa_tailnum_df = spark.sql("""
    select
        *,
        case
            when 
               tailnum is null or tailnum like '' then 'M'
            when
                length(tailnum) != 5 then 'S'
            when
                 substring(tailnum, 1, 1) not like 'N'
                     and
                     substring(tailnum, -1, -1) not like 'Z'
                     and
                 substring(tailnum, 2, 4) rlike '^([^0-9]*)$'
                then 'F'
            when 
                 substring(tailnum, 1, 1) = 'I' 
                     or
                 substring(tailnum, 1, 1) = 'O'
                     or
                 substring(tailnum, 1, 1) = '0' 
                then 'FE'
            when
                substring(tailnum, 1, 1) != 'I'
                then 'FN'
            else
               tailnum
        end as qa_tailnum
    from planes_view
    """)
qa_tailnum_df.show()
#lets save it
#qa_tailnum_df.write.parquet("output/qa_tailnum.parquet")
#we can check it too
tailnum_data = spark.read.parquet("output/qa_tailnum.parquet")
tailnum_data.show()

planes_df = planes_df.withColumn("qa_tailnum", 
                when((col("tailnum") == "")      |
                    (col("tailnum").isNull())   |
                    (col('tailnum').rlike('\t') |
                    (col('tailnum').rlike(' +'))), "M")\
                .when((length("tailnum") != 5), "S")\
                .when(
                    (col('tailnum').substr(1, 1) != 'N')   &
                    (col('tailnum').substr(-1, -1) != 'Z') & 
                    (col('tailnum').substr(2,4).rlike('^([^0-9]*)$')), "F")\
                .when((col("tailnum").substr(1,1) == "I")  |
                    (col("tailnum").substr(1,1) == "O") |
                    (col("tailnum").substr(1,1) == "0"), "FE")\
                .when((col("tailnum").substr(1,1) != "N"),  "FN")\
                .otherwise(col('tailnum'))
            )

planes_df.select('qa_tailnum').show()

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+----------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_tailnum|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+----------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|         S|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|         S|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|         S|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|         S|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|         S|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|         S|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  18

De

In [15]:
#with sql
qa_year_df = spark.sql("""
    select
        *,
        case
            when 
               year is null or year like '' then 'M'
            when
                year < 1950 then 'I'
            else
               year
        end as qa_year
    from planes_view
    """)
qa_year_df.show()

planes_df = planes_df.withColumn("qa_year", 
                when((col("year") == "")      |
                    (col("year").isNull())   |
                    (col('year').rlike('\t') |
                    (col('year').rlike(' +'))), "M")\
                .when((col("year") < 1950), "I")\
                .otherwise(col('year'))
            )
planes_df.select('qa_year').show()

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+-------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_year|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+-------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|   1998|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|   1999|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|   1999|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|   1999|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|   1999|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|   1999|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|   1999|


Novo

In [16]:
type_categories = ['Fixed wing multi engine', 'fixed wing single engine', 'Rotorcraft']

#with sql
qa_type_df = spark.sql("""
    select
        *,
        case
            when 
               type is null or type like '' then 'M'
            when
               type not in ('Fixed wing multi engine', 'fixed wing single engine', 'Rotorcraft') 
               then 'C'
            else
               type
        end as qa_type
    from planes_view
    """)
qa_type_df.show()

planes_df = planes_df.withColumn("qa_type", 
                when((col("type") == "")      |
                    (col("type").isNull())   |
                    (col('type').rlike('\t') |
                    (col('type').rlike(' +'))), "M")\
                .when((col("type").isin(type_categories) == False),  "C")\
                .otherwise(col('type'))
            )

planes_df.select('qa_type').show()

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------------------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|             qa_type|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------------------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|Fixed wing multi ...|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|Fixed wing multi ...|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|Fixed wing multi ...|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|Fixed wing multi ...|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|Fixed wing multi ...|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|Fixed

Que

In [17]:
manufacture_categories = ["AIRBUS", "BOEING","BOMBARDIER","CESSNA","EMBRAER","SIKORSKY","CANADAIR",
                          "PIPER","MCDONNELL DOUGLAS","CIRRUS","BELL","KILDALL GARY","LAMBERT RICHARD",
                          "BARKER JACK","ROBINSON HELICOPTER","GULFSTREAM","MARZ BARRY"]
#with sql
qa_manufacturer_df = spark.sql("""
    select
        *,
        case
            when 
               manufacturer is null or manufacturer like '' then 'M'
            when
               manufacturer not in ("%AIRBUS%", "BOEING","BOMBARDIER","CESSNA","EMBRAER","SIKORSKY","CANADAIR",
                                   "PIPER","MCDONNELL DOUGLAS","CIRRUS","BELL","KILDALL GARY","LAMBERT RICHARD%",
                                   "BARKER JACK","ROBINSON HELICOPTER","GULFSTREAM","MARZ BARRY") 
               then 'C'
            else
               manufacturer
        end as qa_manufacturer
    from planes_view
    """)
qa_manufacturer_df.show()

#using a udf
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType


# dica do eugênio -> df.map(lambda x: any([x.contains(f"%{y}%") for y in MANUFACTURERS]))

@udf
def qa_manufacturer(el):
    if(any([el.__contains__(f"{y}") for y in manufacture_categories])):
        return el
    else:
        return "M"

planes_df.select("manufacturer", qa_manufacturer('manufacturer')).show()

#lets go back to our df

planes_df = planes_df.withColumn("qa_manufacturer", 
                when((col("manufacturer") == "")      |
                    (col("manufacturer").isNull()), "M")\
                .when(
                    (col("manufacturer").contains("AIRBUS%")) |
                    (col("manufacturer").contains("BOEING%")) |
                    (col("manufacturer").contains("BOMBARDIER%")) |
                    (col("manufacturer").contains("CESSNA%"))   |
                    (col("manufacturer").contains("EMBRAER%"))  |
                    (col("manufacturer").contains("SIKORSKY%")) |
                    (col("manufacturer").contains("CANADAIR%")) |
                    (col("manufacturer").contains("PIPER%"))    |
                    (col("manufacturer").contains("MCDONNELL DOUGLAS%")) |
                    (col("manufacturer").contains("CIRRUS%")) |
                    (col("manufacturer").contains("BELL%"))   |
                    (col("manufacturer").contains("KILDALL GARY%"))    |
                    (col("manufacturer").contains("LAMBERT RICHARD%")) |
                    (col("manufacturer").contains("BARKER JACK%"))     |
                    (col("manufacturer").contains("ROBINSON HELICOPTER%")) |
                    (col("manufacturer").contains("GULFSTREAM%")) |
                    (col("manufacturer").contains("MARZ BARRY%")), "C"
                    )\
                .otherwise(col('manufacturer'))
            )
planes_df.select('qa_manufacturer').show()

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+---------------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_manufacturer|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+---------------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|              C|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|              C|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|              C|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|              C|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|              C|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|              C|
| N109UW|1999|Fixed wing mul

Boa

In [18]:
#with sql
qa_model_df = spark.sql("""
    select
        *,
        case
            when 
               model is null or model like '' then 'M'
            when
                 manufacturer = 'AIRBUS'
                     and
                 substring(model, 1, 1) != 'A'
                then 'F'
            when
                 manufacturer = 'BOEING'
                     and
                 substring(model, 1, 1) != '7'
                then 'F'
            when
                 (manufacturer = 'BOMBARDIER' or manufacturer = 'CANADAIR')
                     and
                 substring(model, 1, 1) != 'CL'
                then 'F'
            when
                 manufacturer = 'MCDONNELL DOUGLAS'
                     and
                (
                 substring(model, 1, 1) != 'MD' or
                 substring(model, 1, 1) != 'DC'
                 )
                then 'F'
            else
               model
        end as qa_model
    from planes_view
    """)
qa_model_df.show()

planes_df = planes_df.withColumn("qa_model", 
                when((col("model") == "")     |
                    (col("model").isNull())   |
                    (col('model').rlike('\t') |
                    (col('model').rlike(' +'))), "M")\
                .when(
                    (col('manufacturer') == "AIRBUS") &
                    (col('model').substr(1, 1) != 'A'), "F")\
                .when(
                    (col('manufacturer') == "BOEING") &
                    (col('model').substr(1, 1) != '7'), "F")\
                .when(
                    ((col('manufacturer') == "BOMBARDIER") |
                    (col('manufacturer') == "CANADAIR"))   &
                    (col('model').substr(1, 1) != 'CL'), "F")\
                .when(
                    (col('manufacturer') == "MCDONNELL DOUGLAS") &
                    ((col('model').substr(1, 1) != 'MD') |
                    (col('model').substr(1, 1) != 'DC')), "F")\
                .otherwise(col('model'))
            )
planes_df.select('qa_model').show()

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_model|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|A320-214|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|A320-214|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|A320-214|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|A320-214|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|A320-214|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|A320-214|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|

Surpresa

In [19]:
#with sql
qa_engines_df = spark.sql("""
    select
        *,
        case
            when 
               engines is null or engines like '' then 'M'
            when
                engines < 1 or engines > 4 then 'S'
            when
                engines rlike '^([^0-9]*)$' = False
                then 'A'
            else
               engines
        end as qa_engines
    from planes_view
    """)
qa_engines_df.show()

planes_df = planes_df.withColumn("qa_engines", 
                when((col("engines") == "") |
                    (col("engines").isNull()), "M")\
                .when((col("engines") < 1) |
                    (col("engines") > 4), "I")\
                .when((col('engines').rlike('^[0-9]*$') == False), "A")\
                .otherwise(col('engines'))
            )
planes_df.select('qa_engines').show()

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+----------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_engines|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+----------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|         2|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|         2|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|         2|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|         2|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|         2|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|         2|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  18

Ver você

In [20]:
#with sql
qa_seats_df = spark.sql("""
    select
        *,
        case
            when 
               seats is null or seats like '' then 'M'
            when
                seats < 2 or seats > 500 then 'S'
            when
                 seats rlike '^([^0-9]*)$' = False
                then 'F'
            else
               seats
        end as qa_seats
    from planes_view
    """)
qa_seats_df.show()

planes_df = planes_df.withColumn("qa_seats", 
                when((col("seats") == "")      |
                    (col("seats").isNull()), "M")\
                .when((col("seats") < 2) |
                    (col('seats') > 500), "S")\
                .when((col('seats').rlike('^([^0-9]*)$') == False), "F")\
                .otherwise(col('seats'))
            )
planes_df.show()

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_seats|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|     182|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|     182|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|     182|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|     182|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|     182|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|     182|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|

por

In [21]:
#with sql
qa_speed_df = spark.sql("""
    select
        *,
        case
            when 
               speed is null or speed like '' then 'M'
            when
                speed < 50.0 or speed > 150.0 then 'S'
            when
                 speed rlike '^([^0-9]*)$' = False
                then 'F'
            else
               speed
        end as qa_speed
    from planes_view
    """)
qa_speed_df.show()

planes_df = planes_df.withColumn("qa_speed", 
                when((col("speed") == "")      |
                    (col("speed").isNull()), "M")\
                .when((col("speed") < 50.0) |
                    (col('speed') > 150.0), "S")\
                .when((col('speed').rlike('^([^0-9]*)$') == False), "F")\
                .otherwise(col('speed'))
            )
planes_df.select('qa_speed').show()

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_speed|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|       M|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|       M|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|       M|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|       M|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|       M|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|       M|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|

aqui

In [22]:
engine_categories = ["Turbo-fan", "Turbo-jet","Turbo-prop","Turbo-prop","4 Cycle"]

#with sql
qa_engine_df = spark.sql("""
    select
        *,
        case
            when 
               engine is null or engine like '' then 'M'
            when
               engine not in ("Turbo-fan", "Turbo-jet","Turbo-prop","Turbo-prop","4 Cycle") 
               then 'C'
            else
               engine
        end as qa_engine
    from planes_view
    """)
qa_engine_df.show()

planes_df = planes_df.withColumn("qa_engine", 
                when((col("engine") == "")      |
                    (col("engine").isNull()), "M")\
                .when(
                    (col("engine").contains("Turbo-fan%")) |
                    (col("engine").contains("Turbo-jet%")) |
                    (col("engine").contains("Turbo-prop%")) |
                    (col("engine").contains("Turbo-shaft%"))   |
                    (col("engine").contains("Cycle%")), "C"
                    )\
                .otherwise(col('engine'))
            )
planes_df.select('qa_engine').show()

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+---------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_engine|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+---------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|Turbo-fan|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|Turbo-fan|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|Turbo-fan|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|Turbo-fan|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|Turbo-fan|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|Turbo-fan|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|T

In [63]:
#Após feitas as transformações, vamos salvar nosso arquivo
planes_df.write.parquet("qa_outputs/planes_qa.parquet")
planes_df.show()

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+----------+-------+-------+----------------+--------+----------+--------+--------+---------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_tailnum|qa_year|qa_type| qa_manufacturer|qa_model|qa_engines|qa_seats|qa_speed|qa_engine|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+----------+-------+-------+----------------+--------+----------+--------+--------+---------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|         S|   1998|      M|AIRBUS INDUSTRIE|A320-214|         2|       F|       M|Turbo-fan|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|         S|   1999|      M|AIRBUS INDUSTRIE|A320-214|         2|       F|       M|Turbo-fan|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null

e por fim, mas não menos importante

# Flights Dataset

![Flight](attachment:./images/flight_snapshot.png "Flyng plane")
<p style="text-align: center;"> <a href="./glossary/flights_glossary.txt"> Flight Glossary </a> </p>

Para este último dataset não faremos nada diferete:

In [24]:
#vamos configura-lo

flights_file = "./datasets/flights.csv"

#set our schema (that you can see on glossary)
schema = StructType([
    StructField("year",  IntegerType()),
    StructField("month", IntegerType()),
    StructField("day",   IntegerType()),
    StructField("dep_time",  IntegerType()),
    StructField("dep_delay", IntegerType()),
    StructField("arr_time",  IntegerType()),
    StructField("arr_delay", IntegerType()),
    StructField("carrier",   StringType()),
    StructField("tailnum",   StringType()),
    StructField("flight",  IntegerType()),
    StructField("origin",  StringType()),
    StructField("destiny", StringType()),
    StructField("air_time", IntegerType()),
    StructField("distance", IntegerType()),
    StructField("hour",    IntegerType()),
    StructField("minute",  IntegerType())
])

#and create a new dataframe 
flights_df = spark \
           .read.format("csv") \
           .option("inferSchema", "false") \
           .option("header", "True")\
           .schema(schema)\
           .csv(flights_file)

#lets take the rdd to work with too
flights_rdd = flights_df.rdd

#don't forget to create a view (important to work with spark sql)
flights_df.createOrReplaceTempView('flights_view')

flights_df.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|destiny|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA|    LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA|    HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA|    SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX|    SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA|    BUR|     127|     937|   7|    54|
|2014|    1| 15|

Para finalizar nosso trabalho, neste dataset temos tarefas a realizar que estão descritas a seguir:


O que

In [66]:
#with sql
qa_year_month_day_df = spark.sql("""
    select
        *,
        case
            when 
               year is null or year like '' then 'MY'
            when 
               month is null or month like '' then 'MM'
            when 
               day is null or day like '' then 'MD'
            when
                year < 1950 then 'IY'
            when
                month < 1 or month > 12 then "IM"
            when
                (month == 2 and (day < 1 or day > 29)) or (month != 2 and (day < 1 or day > 31))
                then 'ID'
        end as qa_year_month_day
    from flights_view
    """)
qa_year_month_day_df.show()

flights_df = flights_df.withColumn("qa_year_month_day",
                when((col('year').isNull()) | 
                    ((col('year') == '')), 'MY')\
                .when( (col('month').isNull()) | 
                    (col('month') == ''), 'MM')\
                .when( (col('day').isNull()) | 
                    (col('day') == ''), 'MD')\
                .when((col('year') < 1950 ), "IY")\
                .when((col('month') < 1 ) |
                    (col('month') > 12 ), "IM")\
                .when((
                    (col('month') == 2) &
                    ((col('day') < 1 ) |
                    (col('day') > 29 ))
                    ) |
                    ((col('month') != 2) &
                    ((col('day') < 1 ) |
                    (col('day') > 31 ))
                    ), "ID")\
            )
flights_df.select('qa_year_month_day').show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+-----------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|destiny|air_time|distance|hour|minute|qa_year_month_day|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+-----------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA|    LAX|     132|     954|   6|    58|             null|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA|    HNL|     360|    2677|  10|    40|             null|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA|    SFO|     111|     679|  14|    43|             null|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX|    SJC|      83|     569|  17|     5|             null|
|2014|    3| 

Está

In [26]:
#with sql
qa_hour_minute_df = spark.sql("""
    select
        *,
        case
            when 
               carrier is null or carrier like '' then 'M'
            when
                length(carrier) != 2 then 'F'
            else
                carrier
        end as qa_hour_minute
    from flights_view
    """)
qa_hour_minute_df.show()

flights_df = flights_df.withColumn("qa_hour_minute",
                when((col('hour').isNull()) | 
                    ((col('hour') == '')), 'MH')\
                .when( (col('minute').isNull()) | 
                    (col('minute') == ''), 'MM')\
                .when( (length('hour') == 1) & ( (col('hour').substr(1,1) < 0) | (col('hour').substr(1,1) > 24) ) |
                    (length('hour') == 2) & ((col('hour').substr(1,2) < 0) | (col('hour').substr(1,2) > 24) ), 'IH')\
                .when( ((length('minute') == 1) & ((col('hour').substr(2,3) < 0) | (col('hour').substr(2,3) > 9))) |
                    ((length('minute') == 2) & ((col('hour').substr(3,4) < 0) | (col('hour').substr(3,4) > 59) )), 'IM')\
            )
flights_df.select('qa_hour_minute').show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+--------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|destiny|air_time|distance|hour|minute|qa_hour_minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+--------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA|    LAX|     132|     954|   6|    58|            VX|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA|    HNL|     360|    2677|  10|    40|            AS|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA|    SFO|     111|     679|  14|    43|            VX|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX|    SJC|      83|     569|  17|     5|            WN|
|2014|    3|  9|     754|       -1

achando

In [27]:
#with sql
qa_dep_arr_time_df = spark.sql("""
    select
        *,
        case
            when 
               carrier is null or carrier like '' then 'M'
            when
                length(carrier) != 2 then 'F'
            else
                carrier
        end as qa_dep_arr_delay
    from flights_view
    """)
qa_dep_arr_time_df.show()

flights_df = flights_df.withColumn("qa_dep_arr_time",
                when((col("dep_time") == "") |
                    (col("dep_time").isNull()), "MD")\
                .when((col("arr_time") == "") |
                    (col("arr_time").isNull()), "MA")
                .when( (length('dep_time') == 3) & ( (col('dep_time').substr(1,1) < 0) | (col('dep_time').substr(1,1) > 24) ) |
                    (length('dep_time') == 4) & ((col('dep_time').substr(1,2) < 0) | (col('dep_time').substr(1,2) > 24) ), 'FD')\
                .when( ((length('arr_time') == 3) & ((col('arr_time').substr(2,3) < 0) | (col('arr_time').substr(2,3) > 59))) |
                    ((length('arr_time') == 4) & ((col('arr_time').substr(3,4) < 0) | (col('arr_time').substr(3,4) > 59) )), 'FA')\
            )
flights_df.select('qa_dep_arr_time').show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+----------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|destiny|air_time|distance|hour|minute|qa_dep_arr_delay|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+----------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA|    LAX|     132|     954|   6|    58|              VX|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA|    HNL|     360|    2677|  10|    40|              AS|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA|    SFO|     111|     679|  14|    43|              VX|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX|    SJC|      83|     569|  17|     5|              WN|
|2014|    3|  9|    

do

In [28]:
#with sql
qa_dep_arr_delay_df = spark.sql("""
    select
        *,
        case
            when 
               dep_delay is null or dep_delay like '' then 'MD'
            when
                arr_delay is null or arr_delay like '' then 'MA'
        end as qa_dep_arr_delay
    from flights_view
    """)
qa_dep_arr_delay_df.show()

flights_df = flights_df.withColumn("qa_dep_arr_delay", 
                when((col("dep_delay") == "") |
                    (col("dep_delay").isNull()), "MD")\
                .when((col("arr_delay") == "") |
                    (col("arr_delay").isNull()), "MA")
            )
flights_df.select('qa_dep_arr_delay').show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+----------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|destiny|air_time|distance|hour|minute|qa_dep_arr_delay|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+----------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA|    LAX|     132|     954|   6|    58|            null|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA|    HNL|     360|    2677|  10|    40|            null|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA|    SFO|     111|     679|  14|    43|            null|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX|    SJC|      83|     569|  17|     5|            null|
|2014|    3|  9|    

código

In [29]:
#with sql
qa_carrier_df = spark.sql("""
    select
        *,
        case
            when 
               carrier is null or carrier like '' then 'M'
            when
                length(carrier) != 2 then 'F'
            else
                carrier
        end as qa_carrier
    from flights_view
    """)
qa_carrier_df.show()

flights_df = flights_df.withColumn("qa_carrier", 
                when((col("carrier") == "") |
                    (col("carrier").isNull()), "M")\
                .when((length("carrier") != 2), "F")\
                .otherwise(col('carrier'))
            )
flights_df.select('qa_carrier').show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+----------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|destiny|air_time|distance|hour|minute|qa_carrier|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+----------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA|    LAX|     132|     954|   6|    58|        VX|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA|    HNL|     360|    2677|  10|    40|        AS|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA|    SFO|     111|     679|  14|    43|        VX|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX|    SJC|      83|     569|  17|     5|        WN|
|2014|    3|  9|     754|       -1|    1015|        1|     AS|

até

In [30]:
#with sql
qa_tailnum_df = spark.sql("""
    select
        *,
        case
            when 
               tailnum is null or tailnum like '' then 'M'
            when
                length(tailnum) != 5 then 'S'
            when
                 substring(tailnum, 1, 1) not like 'N'
                     and
                     substring(tailnum, -1, -1) not like 'Z'
                     and
                 substring(tailnum, 2, 4) rlike '^([^0-9]*)$'
                then 'F'
            when 
                 substring(tailnum, 1, 1) = 'I' 
                     or
                 substring(tailnum, 1, 1) = 'O'
                     or
                 substring(tailnum, 1, 1) = '0' 
                then 'FE'
            when
                substring(tailnum, 1, 1) != 'I'
                then 'FN'
            else
               tailnum
        end as qa_tailnum
    from flights_view
    """)
qa_tailnum_df.show()

planes_df = planes_df.withColumn("qa_tailnum", 
                when((col("tailnum") == "")      |
                    (col("tailnum").isNull())   |
                    (col('tailnum').rlike('\t') |
                    (col('tailnum').rlike(' +'))), "M")\
                .when((length("tailnum") != 5), "S")\
                .when(
                    (col('tailnum').substr(1, 1) != 'N')   &
                    (col('tailnum').substr(-1, -1) != 'Z') & 
                    (col('tailnum').substr(2,4).rlike('^([^0-9]*)$')), "F")\
                .when((col("tailnum").substr(1,1) == "I")  |
                    (col("tailnum").substr(1,1) == "O") |
                    (col("tailnum").substr(1,1) == "0"), "FE")\
                .when((col("tailnum").substr(1,1) != "N"),  "FN")\
                .otherwise(col('tailnum'))
            )
planes_df.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+----------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|destiny|air_time|distance|hour|minute|qa_tailnum|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+----------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA|    LAX|     132|     954|   6|    58|         S|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA|    HNL|     360|    2677|  10|    40|         S|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA|    SFO|     111|     679|  14|    43|         S|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX|    SJC|      83|     569|  17|     5|         S|
|2014|    3|  9|     754|       -1|    1015|        1|     AS|

o

In [31]:
#with sql
qa_flight_df = spark.sql("""
    select
        *,
        case
            when 
               flight is null or flight like '' then 'M'
            when
                flight rlike '([0-9]{4})' = False
                then 'F'
            else
               flight
        end as qa_flight
    from flights_view
    """)
qa_flight_df.show()

flights_df =flights_df.withColumn("qa_flight", 
                when((col("flight") == "") |
                    (col("flight").isNull()), "M")\
                .when((col('flight').rlike('[0-9]{4}') == False), "F")\
                .otherwise(col('flight'))
            )
flights_df.select('qa_flight').show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+---------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|destiny|air_time|distance|hour|minute|qa_flight|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+---------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA|    LAX|     132|     954|   6|    58|     1780|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA|    HNL|     360|    2677|  10|    40|      851|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA|    SFO|     111|     679|  14|    43|      755|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX|    SJC|      83|     569|  17|     5|      344|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS

momento,

In [32]:
#with sql
qa_origin_dest_df = spark.sql("""
    select
        *,
        case
            when 
               origin is null or origin like '' then 'M'
            when 
               destiny is null or destiny like '' then 'M'
            when
                origin rlike '([A-Z]|[a-z]|[0-9]{3})' = False or length(origin) != 3
                then 'FO'
            when
                destiny rlike '([A-Z]|[a-z]|[0-9]{3})' = False or length(destiny) != 3
                then 'FD'
            end as qa_origin_dest
    from flights_view
    """)
qa_origin_dest_df.show()

flights_df = flights_df.withColumn("qa_origin_dest", 
                when((col("origin") == "") |
                    (col("origin").isNull()), "MO")\
                .when((col("destiny") == "") |
                    (col("destiny").isNull()), "MD")\
                .when(((col('origin').rlike('([A-Z]|[a-z]|[0-9]{3})') == False) | (length('origin') != 3)), "FO")\
                .when(((col('destiny').rlike('([A-Z]|[a-z]|[0-9]{3})') == False) | (length('destiny') != 3)), "FD")\
            )
flights_df.select('qa_origin_dest').show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+--------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|destiny|air_time|distance|hour|minute|qa_origin_dest|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+--------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA|    LAX|     132|     954|   6|    58|          null|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA|    HNL|     360|    2677|  10|    40|          null|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA|    SFO|     111|     679|  14|    43|          null|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX|    SJC|      83|     569|  17|     5|          null|
|2014|    3|  9|     754|       -1

meu

In [33]:
#with sql
qa_air_time_df = spark.sql("""
    select
        *,
        case
            when 
               air_time is null or air_time like '' then 'M'
            when
                air_time < 20 or air_time > 500 then 'I'
            else
               air_time
        end as qa_air_time
    from flights_view
    """)
qa_air_time_df.show()

flights_df = flights_df.withColumn("qa_air_time", 
                when((col("air_time") == "") |
                    (col("air_time").isNull()), "M")\
                .when((col("air_time") < 20) |
                    (col("air_time") > 500), "I")\
                .otherwise(col('air_time'))
            )
flights_df.select('qa_air_time').show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+-----------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|destiny|air_time|distance|hour|minute|qa_air_time|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+-----------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA|    LAX|     132|     954|   6|    58|        132|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA|    HNL|     360|    2677|  10|    40|        360|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA|    SFO|     111|     679|  14|    43|        111|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX|    SJC|      83|     569|  17|     5|         83|
|2014|    3|  9|     754|       -1|    1015|        1| 

caro

In [34]:
#with sql
qa_distance_df = spark.sql("""
    select
        *,
        case
            when 
               distance is null or distance like '' then 'M'
            when
                distance < 50 or distance > 3000 then 'I'
            else
               distance
        end as qa_distance
    from flights_view
    """)
qa_distance_df.show()

flights_df =flights_df.withColumn("qa_distance", 
                when((col("distance") == "") |
                    (col("distance").isNull()), "M")\
                .when((col("distance") < 50) |
                    (col("distance") > 3000), "I")\
                .otherwise(col('distance'))
            )
flights_df.select('qa_distance').show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+-----------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|destiny|air_time|distance|hour|minute|qa_distance|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+-----------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA|    LAX|     132|     954|   6|    58|        954|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA|    HNL|     360|    2677|  10|    40|       2677|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA|    SFO|     111|     679|  14|    43|        679|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX|    SJC|      83|     569|  17|     5|        569|
|2014|    3|  9|     754|       -1|    1015|        1| 

curioso?

In [35]:
#with sql
qa_airtime_df = spark.sql("""
    select
        *,
        case
            when 
               air_time is null or air_time like '' then 'M'
            when
                air_time >= ( (distance * 0.1) + 30) then 'TL'
            when
                air_time <= ( (distance * 0.1) + 10) then 'TS'
            when
                (
                    (air_time >= ( (distance * .1) + 30)) and
                    (air_time <= ( (distance * .1) + 10))
                ) = False then 'TR'
        end as qa_airtime
    from flights_view
    """)
qa_airtime_df.show()

flights_df =flights_df.withColumn("qa_airtime", 
                when((col("air_time") == "") |
                    (col("air_time").isNull()), "M")\
                .when((col("air_time") >= (col('distance') * .1 ) + 30), "TL")\
                .when((col("air_time") <= (col('distance') * .1 ) + 10), "TS")\
                .when( ((col("air_time") >= (col('distance') * .1 ) + 30) &
                        ((col("air_time") <= (col('distance') * .1 ) + 10))) == False, "TR")\
            )

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+----------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|destiny|air_time|distance|hour|minute|qa_airtime|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+----------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA|    LAX|     132|     954|   6|    58|        TL|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA|    HNL|     360|    2677|  10|    40|        TL|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA|    SFO|     111|     679|  14|    43|        TL|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX|    SJC|      83|     569|  17|     5|        TR|
|2014|    3|  9|     754|       -1|    1015|        1|     AS|

In [57]:
data = flights_df.withColumn('qa_dep_arr_delay', (
    F.when(
        ( (F.col('qa_dep_arr_delay').isNull())
        | (check_empty_column('qa_dep_arr_delay'))
        | (F.col('qa_dep_arr_delay') == None)), 'M'
    ).otherwise(
        F.col('qa_dep_arr_delay')
    )
))

data.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+-----------------+--------------+---------------+----------------+----------+---------+--------------+-----------+-----------+----------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|destiny|air_time|distance|hour|minute|qa_year_month_day|qa_hour_minute|qa_dep_arr_time|qa_dep_arr_delay|qa_carrier|qa_flight|qa_origin_dest|qa_air_time|qa_distance|qa_airtime|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-------+--------+--------+----+------+-----------------+--------------+---------------+----------------+----------+---------+--------------+-----------+-----------+----------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA|    LAX|     132|     954|   6|    58|             null|          null|           null|            null|        VX|     1780|          n

In [67]:
flights_df.write.parquet('qa_outputs/qa_flights')

In [36]:
# joining_dfs = airport_df.join(flights_df, ((airport_df.faa == flights_df.origin)), 'left')\
#                         .join(planes_df, (planes_df.tailnum == flights_df.tailnum), 'left')

In [37]:
# joining_dfs = joining_dfs.drop('tailnum', 'year')

In [38]:
joining_dfs.count()

11395

In [39]:
# joining_dfs.write.parquet('outputs1/joined_data_new.parquet')

AnalysisException: AnalysisException: path file:/data/notebook_files/outputs1/joined_data_new.parquet already exists.

In [36]:
# airport_qa = airport_df.drop('name', 'lat', 'lon', 'alt', 'tz', 'dst')
# planes_qa = planes_df.drop('year', 'type', 'manufacturer', 'model', 'engines', 'seats', 'speed', 'engine')
# flights_qa = flights_df.drop('year', 'month', 'day', 'dep_time', 'dep_delay', 'arr_time', 'arr_delay', 'carrier', 'flight', 
#                              'air_time', 'distance', 'hour', 'minute')

In [37]:
# from pyspark.sql import functions as F

# BD_qa_1 = flights_df.alias('fli').join(airport_df.alias('air'), 
#                                  F.col('fli.origin') == F.col('air.faa'), 'left')\
#                          .select(F.col("tailnum"),
#                                 F.col('origin'),
#                                F.col('destiny'),
#                                F.col('faa'),
#                                F.col('year_month_day_qa'),
#                                F.col('hour_minute_qa'), 
#                                F.col('dep_arr_time_qa'),
#                                F.col('dep_arr_delay_qa'),
#                                F.col('carrier_qa'),
#                                F.col('tailnum_qa'),
#                                F.col('flight_qa'),
#                                F.col('origin_dest_qa'),
#                                F.col('air_time_qa'),
#                                F.col('distance_qa'),
#                                F.col('distance_airtime_qa'),
#                                F.col('faa_qa').alias('ori_qa_faa'),
#                                F.col('name_qa').alias('ori_qa_name'),
#                                F.col('lat_qa').alias('ori_qa_lat'),
#                                F.col('lon_qa').alias('ori_qa_lon'),
#                                F.col('alt_qa').alias('ori_qa_alt'),
#                                F.col('tz_qa').alias('ori_qa_tz'),
#                                F.col('dst_qa').alias('ori_qa_dst'))
# BD_qa_1.show(5)

In [38]:
#Após feitas as transformações, vamos salvar nosso arquivo
#flights_df.write.parquet("output/flights_qa.parquet")

In [39]:
# all_dfs_join = airport_df.alias('a').join(flights_df.alias('b'),(airport_df['faa'] == flights_df['origin']),'left'
#             )\
#            .join(planes_df.alias('c')).where(planes_df['tailnum'] == flights_df['tailnum'])
           
# df_to_export = all_dfs_join.drop(*('tailnum', 'year'))
# df_to_export.show()

In [40]:

#df_to_export.write.parquet('outputs/all_join.parquet')

E com esse arquivo salvo (que lá em cima já vimos como efetuar a leitura), finalizamos a atividade da semana 2.

Obrigado por ler e interagir até aqui!  <span style="font-size:25px">🖖</span> 
 
![Thanks](attachment:./images/text-1647627553578.png "ThankU")

In [ ]:

# joining_dfs.select('*').where(col('qa_seats').isNotNull()).show()